In [1]:
pip install sentence_transformers

In [2]:
#Importing required libraries
import numpy as np
import pandas as pd
import pickle
import nltk
from sentence_transformers import SentenceTransformer
import torch
from torch.nn import CosineSimilarity
import re

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#Loading the dataset
df = pd.read_csv('/content/FAQ_ML.csv', encoding_errors = 'ignore')

In [5]:
df.head()

,Q,A
0,The approach focuses on unsupervis,"The approach focuses on unsupervised learning,..."
1,Density based anomaly detection,This approach is based on the K-nearest neighb...
2,Support Vector Machine-Based Anomaly Detection,The k-means algorithm can be used which partit...
3,Cross-Validation,Cross-Validation is a statistical method of ev...
4,Bagging(Bootstrap aggregation),It is a homogeneous weak learnersÕ model that...


In [6]:
#Removing all alphanumeric characters
df['new_Q'] = df['Q'].str.replace('[^\w\s]','', regex = False)
df['new_A'] = df['A'].str.replace('[^\w\s]','', regex = False)

In [7]:
pd.set_option('display.max_colwidth', None)
df

,Q,A,new_Q,new_A
0,The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids.",The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids."
1,Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have.",Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have."
2,Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies.",Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies."
3,Cross-Validation,"Cross-Validation is a statistical method of evaluating and comparing learning algorithms by dividing data into two segments: one used to learn or train a model and the other used to validate the model. \n\nIn typical cross-validation, the training and validation sets must cross-over in successive rounds such that each data point has a chance of being validated against. \n\nThe basic form of cross-validation is k-fold cross-validation. Other forms of cross-validation are special cases of k-fold cross-validation or involve repeated rounds of k-fold cross-validation.\n\nIn k-fold cross-validation, the data is first partitioned into k equally (or nearly equally) sized segments or folds. Subsequently k iterations of training and validation are performed such that within each iteration a different fold of the data is held-out for validation while the remaining k ? 1 folds are used for learning. \n\nSo, why do we use cross-validation?\n\nIt allows us to get more metrics and draw important conclusion about our algorithm and our data.\n\nHelps to tune the hyper parameters of a given machine learning algorithm, to get good performance according to some suitable metric.\n\nIt mitigates overfitting while building a pipeli

In [8]:
df['new_Q'] = df['Q'].str.lower()
df['new_A'] = df['A'].str.lower()
df

,Q,A,new_Q,new_A
0,The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids.",the approach focuses on unsupervis,"the approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids."
1,Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have.",density based anomaly detection,"this approach is based on the k-nearest neighbors algorithm. itõs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. to measure the nearest set of a data point, you can use euclidean distance or similar measure according to the type of data you have."
2,Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies.",support vector machine-based anomaly detection,"the k-means algorithm can be used which partition the dataset into a gia support vector machine is another effective technique for detecting anomalies. one-class svms have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nthis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nanomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. this rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. any data points that fall outside of these clusters are considered as anomalies."
3,Cross-Validation,"Cross-Validation is a statistical method of evaluating and comparing learning algorithms by dividing data into two segments: one used to learn or train a model and the other used to validate the model. \n\nIn typical cross-validation, the training and validation sets must cross-over in successive rounds such that each data point has a chance of being validated against. \n\nThe basic form of cross-validation is k-fold cross-validation. Other forms of cross-validation are special cases of k-fold cross-validation or involve repeated rounds of k-fold cross-validation.\n\nIn k-fold cross-validation, the data is first partitioned into k equally (or nearly equally) sized segments or folds. Subsequently k iterations of training and validation are performed such that within each iteration a different fold of the data is held-out for validation while the remaining k ? 1 folds are used for learning. \n\nSo, why do we use cross-validation?\n\nIt allows us to get more metrics and draw important conclusion about our algorithm and our data.\n\nHelps to tune the hyper parameters of a given machine learning algorithm, to get good performance according to some suitable metric.\n\nIt mitigates overfitting while building a pipeli

In [9]:
#Removing Punctuation, less than, greater than, question mark & Digits
df['new_Q'] = df['Q'].str.replace('<', '').str.replace('>', '').str.replace('?', '',regex = False).str.replace('\d', '',regex = False)
df['new_A'] = df['A'].str.replace('<', '').str.replace('>', '').str.replace('?', '',regex = False).str.replace('\d', '',regex = False)
df

,Q,A,new_Q,new_A
0,The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids.",The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids."
1,Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have.",Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have."
2,Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies.",Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies."
3,Cross-Validation,"Cross-Validation is a statistical method of evaluating and comparing learning algorithms by dividing data into two segments: one used to learn or train a model and the other used to validate the model. \n\nIn typical cross-validation, the training and validation sets must cross-over in successive rounds such that each data point has a chance of being validated against. \n\nThe basic form of cross-validation is k-fold cross-validation. Other forms of cross-validation are special cases of k-fold cross-validation or involve repeated rounds of k-fold cross-validation.\n\nIn k-fold cross-validation, the data is first partitioned into k equally (or nearly equally) sized segments or folds. Subsequently k iterations of training and validation are performed such that within each iteration a different fold of the data is held-out for validation while the remaining k ? 1 folds are used for learning. \n\nSo, why do we use cross-validation?\n\nIt allows us to get more metrics and draw important conclusion about our algorithm and our data.\n\nHelps to tune the hyper parameters of a given machine learning algorithm, to get good performance according to some suitable metric.\n\nIt mitigates overfitting while building a pipeli

In [10]:
#Removing Stop words
import nltk
from nltk.corpus import stopwords
",".join(stopwords.words('english'))
stopwords = set(stopwords.words('english'))

stop_words = nltk.corpus.stopwords.words('english')

def remove_stop(x):
    return ",".join([word for word in str(x).split() if word not in stop_words])

In [11]:
df['new_Q'] = df['Q'].apply(lambda x : remove_stop(x))
df['new_A'] = df['A'].apply(lambda x : remove_stop(x))
df

,Q,A,new_Q,new_A
0,The approach focuses on unsupervis,"The approach focuses on unsupervised learning, similar data points tend to belong to similar groups or clusters, as determined by their distance from local centroids.","The,approach,focuses,unsupervis","The,approach,focuses,unsupervised,learning,,similar,data,points,tend,belong,similar,groups,clusters,,determined,distance,local,centroids."
1,Density based anomaly detection,"This approach is based on the K-nearest neighbors algorithm. ItÕs evident that normal data points always occur around a dense neighborhood and abnormalities deviate far away. To measure the nearest set of a data point, you can use Euclidean distance or similar measure according to the type of data you have.","Density,based,anomaly,detection","This,approach,based,K-nearest,neighbors,algorithm.,ItÕs,evident,normal,data,points,always,occur,around,dense,neighborhood,abnormalities,deviate,far,away.,To,measure,nearest,set,data,point,,use,Euclidean,distance,similar,measure,according,type,data,have."
2,Support Vector Machine-Based Anomaly Detection,"The k-means algorithm can be used which partition the dataset into a giA support vector machine is another effective technique for detecting anomalies. One-Class SVMs have been devised for cases in which one class only is known, and the problem is to identify anything outside this class.\n\nThis is known as novelty detection, and it refers to automatic identification of unforeseen or abnormal phenomena, i.e. outliers, embedded in a large amount of normal data.\n\nAnomaly detections helps to monitor any data source, including user logs, devices, networks, and servers. This rapidly helps in identifying zero-day attacks as well as unknown security threats.\nven number of clusters. Any data points that fall outside of these clusters are considered as anomalies.","Support,Vector,Machine-Based,Anomaly,Detection","The,k-means,algorithm,used,partition,dataset,giA,support,vector,machine,another,effective,technique,detecting,anomalies.,One-Class,SVMs,devised,cases,one,class,known,,problem,identify,anything,outside,class.,This,known,novelty,detection,,refers,automatic,identification,unforeseen,abnormal,phenomena,,i.e.,outliers,,embedded,large,amount,normal,data.,Anomaly,detections,helps,monitor,data,source,,including,user,logs,,devices,,networks,,servers.,This,rapidly,helps,identifying,zero-day,attacks,well,unknown,security,threats.,ven,number,clusters.,Any,data,points,fall,outside,clusters,considered,anomalies."
3,Cross-Validation,"Cross-Validation is a statistical method of evaluating and comparing learning algorithms by dividing data into two segments: one used to learn or train a model and the other used to validate the model. \n\nIn typical cross-validation, the training and validation sets must cross-over in successive rounds such that each data point has a chance of being validated against. \n\nThe basic form of cross-validation is k-fold cross-validation. Other forms of cross-validation are special cases of k-fold cross-validation or involve repeated rounds of k-fold cross-validation.\n\nIn k-fold cross-validation, the data is first partitioned into k equally (or nearly equally) sized segments or folds. Subsequently k iterations of training and validation are performed such that within each iteration a different fold of the data is held-out for validation while the remaining k ? 1 folds are used for learning. \n\nSo, why do we use cross-validation?\n\nIt allows us to get more metrics and draw important conclusion about our algorithm and our data.\n\nHelps to tune the hyper parameters of a given machine learning algorithm, to get good performance according to some suitable metric.\n\nIt mitigates overfitting while building a pipeline of models, such that secondÕs models input will be real predictions on data that our first model never seen before.\n\nK-fold cross validation also significantly reduces bias as we are using most of the data for fitting, and also significan

In [12]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [13]:
q_embs = model.encode(df['new_Q'])

In [14]:
file = "Data.pkl"

In [15]:
with open(file, 'wb') as files:
    pickle.dump(q_embs, files)

In [16]:
with open(file, 'rb') as files:
    q_embs = pickle.load(files)

In [17]:
usr_query="Hey explain me about K Means Clustering !!"

In [18]:
def remove_digits(text):
    return re.sub("\d", "", text)

usr_query = remove_digits(usr_query)

In [19]:
def pred_ans(usr_query):
    usr_query = remove_digits(usr_query)

    df1_quary = pd.DataFrame([usr_query], index=['usr_query'], columns=['usr_query'])

    df1_quary['clean_usr_q'] = df1_quary['usr_query'].str.replace('[^\w\s]','', regex = False)

    usr_q_emd = model.encode(df1_quary['clean_usr_q'])

    cosine_similarity = CosineSimilarity()

    q_idx = np.argmax(cosine_similarity(torch.from_numpy(usr_q_emd), torch.from_numpy(q_embs)))
    return df['A'][q_idx.item()]

In [20]:
df['new_Q']

0                                    The,approach,focuses,unsupervis
1                                    Density,based,anomaly,detection
2                     Support,Vector,Machine-Based,Anomaly,Detection
3                                                   Cross-Validation
4                                     Bagging(Bootstrap,aggregation)
                                   ...                              
102                  What,role,Activation,functions,Neural,Networks?
103    List,names,popular,Activation,Functions,used,Neural,Networks.
104                                           Machine,Learning,works
105                                   opportunities,machine,learning
106                                  use,Python,Machine,Learning,AI?
Name: new_Q, Length: 107, dtype: object

In [21]:
while True:
    usr_q=input("Ask me a question (or exit for type 'exit'):")
    if(usr_q == "exit"):
        print('Thank You !')
        break
    else:
        print("Answer is :\n", pred_ans(usr_q))
        print("___________________________________________________________________________________________")

Ask me a question (or exit for type 'exit'):Explain me about k means Clustering.
Answer is :
  K-means clustering represents an unsupervised algorithm, mainly used for clustering, while KNN is a supervised learning algorithm used for classification.
___________________________________________________________________________________________
Ask me a question (or exit for type 'exit'):Machine Learning means?
Answer is :
 Machine learning is a type of artificial intelligence
that allows computer systems to automatically
learn from data and improve their performance on
a specific task without being explicitly
programmed. It is based on the idea that
machines can learn from examples and
experience, rather than being explicitly
programmed to perform a specific task.
___________________________________________________________________________________________
Ask me a question (or exit for type 'exit'):Activation functions
Answer is :
 The reason for using activation functions in Neural Network